In [6]:

import cv2 as cv 
import dlib  
import numpy as np  
import tensorflow as tf 
import matplotlib.pyplot as plt  
from scipy.spatial import distance as dist 
#----------------------------------------------ZMIENNE---------------------------------
EYE_AR_THRESH = 0.25  #Próg zamknięcia oczu 
EYE_AR_CONSEC_FRAMES = 20  #Liczba klatek (próg senności) 
blink_counter = 0 #Licznik mrugnięć 
drowsy_alert = False #Flaga ostrzeżenia
prog_ziewania = 0.7

#-------------------------------------------FUNKCJE-------------------------------------
#Funkcja obliczająca Eye Aspect Ratio
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])  #Odległość między górną i dolną powieką (wewnętrzna część oka)
    B = dist.euclidean(eye[2], eye[4])  #Odległość między górną i dolną powieką (zewnętrzna część oka)
    C = dist.euclidean(eye[0], eye[3])  #Odległość między kącikami oka
    return (A + B) / (2.0 * C)  #Wskaźnik EAR - im niższy, tym bardziej zamknięte oczy

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    C = dist.euclidean(mouth[0], mouth[6])   
    return (A + B) / (2.0 * C)             

#--------------------------------------------------------------------------------------

#Inicjalizacja detektora twarzy, modelu predykcji, kamery
detector = dlib.get_frontal_face_detector() 
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
cap = cv.VideoCapture(0) 

#Pobranie klatki obrazu z kamery
while True:
    ret, frame = cap.read()
    if not ret:  
        break

    #Wykrywanie twarzy
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = detector(gray)
                         
    for face in faces:
        #Pobranie punktów charakterystycznych twarzy (oczy, nos, usta)
        landmarks = predictor(gray, face)
        left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
        right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])
        mouth = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)])
        cv.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (0, 0, 255), 2)

        #Obliczenie wskaźnika EAR dla obu oczu, MAR dla ust
        left_EAR = eye_aspect_ratio(left_eye)
        right_EAR = eye_aspect_ratio(right_eye)
        EAR = (left_EAR + right_EAR) / 2.0  # Średnia wartość EAR dla obu oczu
        MAR = mouth_aspect_ratio(mouth)
        
        if EAR <= EYE_AR_THRESH:
            blink_counter += 1 
        else:
            if blink_counter >= EYE_AR_CONSEC_FRAMES:
                drowsy_alert = True
            blink_counter = 0  

        if MAR>=prog_ziewania:
            drowsy_alert = True
        else: 
            drowsy_alert = False

    #Status senności
    cv.putText(frame, f"Sleepy: {drowsy_alert}", (10, 30),
    cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


    cv.imshow("Kamera", frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


In [19]:
!git add .
